Preparamos 

In [3]:
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 9.3.0
-- The CXX compiler identification is GNU 9.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /home/lucas/metodos-tp3-master/tpenv/bin/python (found version "3.8.5") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython3.8.so
-- pybind11 v2.2.4
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/lucas/metodos-tp3-master
-- Configuring done
-- Generat

In [8]:
import metnum
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_squared_log_error, median_absolute_error


!pwd
!python --version

/home/lucas/metodos-tp3-master/notebooks
Python 3.8.5


Importamos los datos

In [11]:
set_original = pd.read_csv("../data/train.csv")
display(set_original)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,23533.0,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,48551.0,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,47835.0,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,BOSQUES,Zinacantepec,Edo. de México,0.0,2.0,2.0,...,53666.0,NaN,NaN,2015-02-08 00:00:00,0.0,0.0,0.0,0.0,0.0,650000.0
239996,259178,casa en condominio a 10 min. del centro de toluca,"casa con un jardin amplio, un cuarto de servic...",Casa,Filiberto Navas 325,Toluca,Edo. de México,0.0,3.0,3.0,...,51954.0,19.294665,-99.692916,2014-07-10 00:00:00,0.0,0.0,0.0,1.0,1.0,1940000.0
239997,131932,nicolas san juan,"departamento con excelente ubicación, muy cerc...",Apartamento,Nicolas San Juan,Benito Juárez,Distrito Federal,20.0,2.0,1.0,...,50003995.0,NaN,NaN,2015-03-03 00:00:00,0.0,0.0,0.0,0.0,0.0,3400000.0
239998,146867,casa sola. javier rojo gomez.,"casa sola, dividida en cuatro departamentos de...",Casa,Javier Rojo Gomez 120,Iztapalapa,Distrito Federal,20.0,4.0,0.0,...,24162.0,19.366651,-99.082246,2014-12-26 00:00:00,1.0,0.0,0.0,1.0,1.0,2890000.0


Veamos las categorias que tenemos:

In [7]:
list(set_original.columns)

['id',
 'titulo',
 'descripcion',
 'tipodepropiedad',
 'direccion',
 'ciudad',
 'provincia',
 'antiguedad',
 'habitaciones',
 'garages',
 'banos',
 'metroscubiertos',
 'metrostotales',
 'idzona',
 'lat',
 'lng',
 'fecha',
 'gimnasio',
 'usosmultiples',
 'piscina',
 'escuelascercanas',
 'centroscomercialescercanos',
 'precio']

In [19]:
set_original["precio"].values

array([2273000., 3600000., 1200000., ..., 3400000., 2890000., 3650000.])

#Funciones necesarias:

In [20]:
def CML_KFold(K, X, Y): # K: cortes en kfold ; X : matriz sin la columna de precios ; Y : columna que le falta a X
    cortes = KFold(n_splits=K) #quiero que esta variable tome C cortes
    cortes.get_n_splits(X) #mandamos nuestro X a nuestra variable que toma K  cortes => Sacamos K cortes de X
    
    rmse = []
    rmsle = []
    i = 0
    for iTrain, iTest in cortes.split(X):
        i += 1
        # Separamos train y test
        Xtrain, Xtest = X[iTrain], X[iTest] 
        YTrain, YTest = Y[iTrain], Y[iTest]
        print(f"\nCorte {i}, ")
        
        # Inicializamos el CML con las funciones implementadas en el TP 
        CML = metnum.LinearRegression()
        
        #fiteamos
        CML.fit(Xtrain, YTrain)
        
        # Predecimos
        pred = CML.predict(Xtest)
        
        # Para evitar errores, vamos a tomar modulo en los precios de pred y Ytest
        Ytest = np.abs(Ytest)
        pred = np.abs(pred)
        
        # Comparo las predicciones con los resultados esperados.
        
        rmse.append(np.sqrt(mean_squared_error(Ytest, pred)))
        rmsle.append(np.sqrt(mean_squared_log_error(Ytest, pred)))

        print("fin.")
        

    return [rmse,rmsle]

In [22]:
def metricas(Y, pred):
    rmse = np.sqrt(mean_squared_error(Y, pred))
    rmsle = np.sqrt(mean_squared_log_error(Y, pred))
    print(f"RMSE = {rmse}")
    print(f"RMSLE = {rmsle}")

In [21]:
def metricasProm(rms):
    rmse = rms[0]
    rmsle = rms[1]
    print(f"RMSE = {(np.mean(rmse), np.std(rmse))}")
    print(f"RMSLE = {(np.mean(rmsle), np.std(rmsle))}")

In [24]:
def cleanOutliers(X,Y,pred,m=2):
    dist = np.abs(Y - pred)
    mean = np.mean(dist)
    std = np.std(dist)
    
    Xclean = []
    Yclean = []
    for i in range(len(Y)):
        if (dist[i] <= mean + m * std):
            Xclean.append(X[i])
            Yclean.append(Y[i])
            
    return [Xclean,Yclean]